# Data Preperation Light
Author: P.C.O. Maseland <br>
Date: 03 December 2021<br>

This ipynb file contains script to merge all seperate csv files into one whole dataset of light sensor.

In [1]:
#Import packages/libraries
import os
import pandas as pd
import csv
from zipfile import ZipFile
import numpy as np

#### Directory / file handling

In [2]:
#opening data files
path = 'Rawdata2/'
dirs = os.listdir(path)

In [3]:
#find folders
for file in dirs:
    print(file)

proband1
proband10
proband11
proband12
proband13
proband14
proband15
proband2
proband3
proband4
proband5
proband6
proband7
proband8
proband9


In [4]:
#create a list with all zip file paths
subpath = ''
count = 0
paths_list = []

for proband in dirs:
    subpath = path + proband
    subdir = os.listdir(subpath)
    for foldername in subdir:
        if foldername == 'data':
            subsubpath = subpath + "/data"
            subsubdir = os.listdir(subsubpath)
            for file in subsubdir:
                newpath = subsubpath + "/" + file
                paths_list.append(newpath)
                count += 1
        else:
            break

In [5]:
#all zip file paths
paths_list

['Rawdata2/proband1/data/acc_climbingdown_csv.zip',
 'Rawdata2/proband1/data/acc_climbingdown_sqlite.zip',
 'Rawdata2/proband1/data/acc_climbingup_csv.zip',
 'Rawdata2/proband1/data/acc_climbingup_sqlite.zip',
 'Rawdata2/proband1/data/acc_jumping_csv.zip',
 'Rawdata2/proband1/data/acc_jumping_sqlite.zip',
 'Rawdata2/proband1/data/acc_lying_csv.zip',
 'Rawdata2/proband1/data/acc_lying_sqlite.zip',
 'Rawdata2/proband1/data/acc_running_csv.zip',
 'Rawdata2/proband1/data/acc_running_sqlite.zip',
 'Rawdata2/proband1/data/acc_sitting_csv.zip',
 'Rawdata2/proband1/data/acc_sitting_sqlite.zip',
 'Rawdata2/proband1/data/acc_standing_csv.zip',
 'Rawdata2/proband1/data/acc_standing_sqlite.zip',
 'Rawdata2/proband1/data/acc_walking_csv.zip',
 'Rawdata2/proband1/data/acc_walking_sqlite.zip',
 'Rawdata2/proband1/data/gps_climbingdown_csv.zip',
 'Rawdata2/proband1/data/gps_climbingdown_sqlite.zip',
 'Rawdata2/proband1/data/gps_climbingup_csv.zip',
 'Rawdata2/proband1/data/gps_climbingup_sqlite.zip',


In [6]:
#remove sqlite.zip files and only use light.zip files
paths_list_light = [x for x in paths_list if "sqlite" not in x and "lig" in x]

#### Zipfile handling
function 'openzip' has as input a zipfile path and SubjectNr. The function extract all csv files from zip, removes the readMe text file and retrieves info from file name:
* Sensory_type
* Activity
* Body-position

As output the function generates a list with csv file's and list of labels that stick with that csv file.

In [7]:
#input zipfilename, output list of csv files
def openzip(file_name, subject_id):
    labellist = []
    
    with ZipFile(file_name, 'r') as zip:
        csvlist = zip.namelist()
    
        if 'readMe' in csvlist:
            csvlist.remove('readMe')

        for name in csvlist:
            sublabellist = []
            zip.extract(name)

            #get labels
            templist = name.split("_")
            sensory_type = templist[0]
            activity = templist[1]
            body_position = templist[2].split(".")
            body_position = body_position[0]

            #create sublist in csvlist per csv file for labels
            sublabellist.append(sensory_type)
            sublabellist.append(activity)
            sublabellist.append(body_position)

            labellist.append(sublabellist)

        return csvlist, labellist

The 'Create_dataframe' function has as input the csvfile name, the according labellist and a count the nr of dataframe that is created. Since we merging per sensor-type, activity and subject the attributes. Meaning that we only need the Subject_ID, sensor-type and activty column 1 time. <br>

Further, the column attributes name will be changed according the body-position the data is from

With as output a dataframe.

In [9]:
def create_dataframe_light(csvfile, labellist, count):
    df = pd.read_csv(csv)
    
    if count == 0:
        #add columns to df using labellist
        df['Subject ID'] = subject_ID
        df['Sensor-type'] = labellist[count][0]
        df['Activity'] = labellist[count][1]
          
    body_position = labellist[count][2]  
        
    df = df.rename(columns=
                {'attr_light': 'attr_light_' + body_position})
    return df

The main code path down below, retrieves for each zip file from which subject it is and generates a csv file containing the merged dataframe of each sensor-type, activity, subject combination. 

In [10]:
for path in paths_list_light:
            #retrieve labels
            templist = path.split("/")
            templist2 = templist[3].split("_")

            #get subject label
            subject_ID = templist[1].replace("proband", '')

            print("PATH: " + path + " SUBJECT: " + subject_ID)

            count = 0
            csvlist, labellist = openzip(path, subject_ID)

            for csv in csvlist:

                if count == 0:
                    #first_df
                    df1 = create_dataframe_light(csv, labellist, count)
                    #print('START dataframe    ' + csv)
                else:
                    #merge
                    df2 = create_dataframe_light(csv, labellist, count)
                    merged_df = df1.merge(df2, on='id')
                    df1 = merged_df
                    #print('MERGE COMPLETE    ' + csv)

                count += 1

                #remove csv after use
                os.remove(csv)  

            #temporary csv
            filename = subject_ID + '_' + labellist[0][0] + '_' + labellist[0][1] + '_' + 'Merged.csv'
            print("MERGE FILE COMPLETE: " + filename)

            merged_df.to_csv(filename, index=True)

PATH: Rawdata2/proband1/data/lig_climbingdown_csv.zip SUBJECT: 1
MERGE FILE COMPLETE: 1_Light_climbingdown_Merged.csv
PATH: Rawdata2/proband1/data/lig_climbingup_csv.zip SUBJECT: 1
MERGE FILE COMPLETE: 1_Light_climbingup_Merged.csv
PATH: Rawdata2/proband1/data/lig_jumping_csv.zip SUBJECT: 1
MERGE FILE COMPLETE: 1_Light_jumping_Merged.csv
PATH: Rawdata2/proband1/data/lig_lying_csv.zip SUBJECT: 1
MERGE FILE COMPLETE: 1_Light_lying_Merged.csv
PATH: Rawdata2/proband1/data/lig_running_csv.zip SUBJECT: 1
MERGE FILE COMPLETE: 1_Light_running_Merged.csv
PATH: Rawdata2/proband1/data/lig_sitting_csv.zip SUBJECT: 1
MERGE FILE COMPLETE: 1_Light_sitting_Merged.csv
PATH: Rawdata2/proband1/data/lig_standing_csv.zip SUBJECT: 1
MERGE FILE COMPLETE: 1_Light_standing_Merged.csv
PATH: Rawdata2/proband1/data/lig_walking_csv.zip SUBJECT: 1
MERGE FILE COMPLETE: 1_Light_walking_Merged.csv
PATH: Rawdata2/proband10/data/lig_climbingdown_csv.zip SUBJECT: 10
MERGE FILE COMPLETE: 10_Light_climbingdown_Merged.csv
P

PATH: Rawdata2/proband4/data/lig_jumping_csv.zip SUBJECT: 4
MERGE FILE COMPLETE: 4_Light_jumping_Merged.csv
PATH: Rawdata2/proband4/data/lig_lying_csv.zip SUBJECT: 4
MERGE FILE COMPLETE: 4_Light_lying_Merged.csv
PATH: Rawdata2/proband4/data/lig_running_csv.zip SUBJECT: 4
MERGE FILE COMPLETE: 4_Light_running_Merged.csv
PATH: Rawdata2/proband4/data/lig_sitting_csv.zip SUBJECT: 4
MERGE FILE COMPLETE: 4_Light_sitting_Merged.csv
PATH: Rawdata2/proband4/data/lig_standing_csv.zip SUBJECT: 4
MERGE FILE COMPLETE: 4_Light_standing_Merged.csv
PATH: Rawdata2/proband4/data/lig_walking_csv.zip SUBJECT: 4
MERGE FILE COMPLETE: 4_Light_walking_Merged.csv
PATH: Rawdata2/proband5/data/lig_climbingdown_csv.zip SUBJECT: 5
MERGE FILE COMPLETE: 5_Light_climbingdown_Merged.csv
PATH: Rawdata2/proband5/data/lig_climbingup_csv.zip SUBJECT: 5
MERGE FILE COMPLETE: 5_Light_climbingup_Merged.csv
PATH: Rawdata2/proband5/data/lig_jumping_csv.zip SUBJECT: 5
MERGE FILE COMPLETE: 5_Light_jumping_Merged.csv
PATH: Rawdata2

In [11]:
#adjust to what sensor you want a file to merge
#make dataset containing all csv's of accuracy
import glob
dir = os.getcwd()
dirs = os.listdir(dir)
sensor = 'Light'

for item in dirs:
    newdirs = glob.glob('*' + sensor + '*.csv') 
            

In [12]:
count = 0
for csv in newdirs:    
    if count == 0:
        df = pd.read_csv(csv)
    else: 
        df2 = pd.read_csv(csv)
        merged_df = df.append(df2)
        df = merged_df
        print(csv)
        print(merged_df.shape)
    count += 1

10_Light_climbingup_Merged.csv
(44903, 17)
10_Light_jumping_Merged.csv
(50091, 17)
10_Light_lying_Merged.csv
(82092, 17)
10_Light_running_Merged.csv
(114307, 17)
10_Light_sitting_Merged.csv
(146432, 17)
10_Light_standing_Merged.csv
(179498, 17)
10_Light_walking_Merged.csv
(211266, 17)
11_Light_climbingdown_Merged.csv
(236165, 17)
11_Light_climbingup_Merged.csv
(267627, 17)
11_Light_jumping_Merged.csv
(272615, 17)
11_Light_lying_Merged.csv
(305372, 17)
11_Light_running_Merged.csv
(336455, 17)
11_Light_sitting_Merged.csv
(367735, 17)
11_Light_standing_Merged.csv
(399212, 17)
11_Light_walking_Merged.csv
(432498, 17)
12_Light_climbingdown_Merged.csv
(456872, 17)
12_Light_climbingup_Merged.csv
(484873, 17)
12_Light_jumping_Merged.csv
(489945, 17)
12_Light_lying_Merged.csv
(521240, 17)
12_Light_running_Merged.csv
(552167, 17)
12_Light_sitting_Merged.csv
(583506, 17)
12_Light_standing_Merged.csv
(614374, 17)
12_Light_walking_Merged.csv
(646121, 17)
13_Light_climbingdown_Merged.csv
(668049, 17

In [13]:
merged_df.head(100)

,Unnamed: 0,id,attr_time_x,attr_light_chest,Subject ID,Sensor-type,Activity,attr_time_y,attr_light_head,attr_time_x.1,attr_light_shin,attr_time_y.1,attr_light_thigh,attr_time_x.2,attr_light_upperarm,attr_time_y.2,attr_light_waist,attr_time
0,0,1,1436802576022,1.0,10,Light,climbingdown,1436802576054,0.0,1436802576031,0.0,1436802576047,0.0,1.436803e+12,25.0,1.436803e+12,0.0,NaN
1,1,2,1436802576043,1.0,10,Light,climbingdown,1436802576059,0.0,1436802576049,0.0,1436802576048,0.0,1.436803e+12,25.0,1.436803e+12,0.0,NaN
2,2,3,1436802576062,1.0,10,Light,climbingdown,1436802576060,0.0,1436802576069,0.0,1436802576049,0.0,1.436803e+12,25.0,1.436803e+12,0.0,NaN
3,3,4,1436802576083,1.0,10,Light,climbingdown,1436802576079,0.0,1436802576090,0.0,1436802576061,0.0,1.436803e+12,25.0,1.436803e+12,0.0,NaN
4,4,5,1436802576102,1.0,10,Light,climbingdown,1436802576100,0.0,1436802576109,0.0,1436802576080,0.0,1.436803e+12,25.0,1.436803e+12,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,96,1436802577923,0.0,10,Light,climbingdown,1436802577921,0.0,1436802577932,0.0,1436802577904,0.0,1.436803e+12,26.0,1.436803e+12,0.0,NaN
96,96,97,1436802578011,0.0,10,Light,climbingdown,1436802577944,0.0,1436802577955,0.0,1436802577920,0.0,1.436803e+12,26.0,1.436803e+12,0.0,NaN
97,97,98,1436802578012,0.0,10,Light,climbingdown,1436802577962,0.0,1436802577975,0.0,1436802577944,0.0,1.436803e+12,26.0,1.436803e+12,0.0,NaN
98,98,99,1436802578013,0.0,10,Light,climbingdown,1436802577980,0.0,1436802577995,0.0,1436802577961,0.0,1.436803e+12,26.0,1.436803e+12,0.0,NaN


In [14]:
merged_df.columns

Index(['Unnamed: 0', 'id', 'attr_time_x', 'attr_light_chest', 'Subject ID',
       'Sensor-type', 'Activity', 'attr_time_y', 'attr_light_head',
       'attr_time_x.1', 'attr_light_shin', 'attr_time_y.1', 'attr_light_thigh',
       'attr_time_x.2', 'attr_light_upperarm', 'attr_time_y.2',
       'attr_light_waist', 'attr_time'],
      dtype='object')

In [15]:
#Drop unneccesary columns
merged_df = merged_df.drop(columns=['Unnamed: 0',
                     'attr_time',
                     'attr_time_x',
                     'attr_time_y',
                     'attr_time_x.1',
                     'attr_time_y.1',
                     'attr_time_x.2',
                     'attr_time_y.2'])

In [16]:
merged_df.columns

Index(['id', 'attr_light_chest', 'Subject ID', 'Sensor-type', 'Activity',
       'attr_light_head', 'attr_light_shin', 'attr_light_thigh',
       'attr_light_upperarm', 'attr_light_waist'],
      dtype='object')

In [17]:
merged_df.to_csv("tempfile_lig.csv")

In [18]:
merged_df.shape

(3263298, 10)

In [19]:
#look at NaN values
merged_df.isnull().sum(axis = 0)

id                        0
attr_light_chest          0
Subject ID                0
Sensor-type               0
Activity                  0
attr_light_head           0
attr_light_shin           0
attr_light_thigh       4743
attr_light_upperarm       0
attr_light_waist          0
dtype: int64